In [1]:
# Install dependencies
!pip install google-generativeai pdfplumber langchain faiss-cpu --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 44.0 MB/s eta 0:00:00


In [2]:
import os
import google.generativeai as genai

# Configure Gemini (enter your API key when running in Colab)
genai.configure(api_key="YOUR_GEMINI_API_KEY")



# Step 1: Upload & Extract Document

In [6]:
from google.colab import files
# Upload PDF or TXT
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print("✅ File uploaded:", file_name)

def extract_text(file_path):
    if file_path.endswith(".pdf"):
        import pdfplumber
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    elif file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    else:
        raise ValueError("Please upload PDF or TXT only.")

doc_text = extract_text(file_name)
print("📖 Preview:\n", doc_text[:600])

Saving transcript_5.txt to transcript_5.txt
✅ File uploaded: transcript_5.txt
📖 Preview:
 Substituting some shooting frozen RNN,\nbut when does the clock\nokay.\nLet's start.\nOkay. So yes.\nWe will indeed discusses and 31 for today.\nI'm not an and is traveling for an academy Conference.\nHe's giving a keynote.\nHe's a keynote speaker and important conference.\nSo he, he's not here today.\nAnd he asked me if I can cover this class for him happily.\nSome of you know me and the right,\nfor others, My name is delete kiloton.\nI'm an Associate Professor in the Departments.\nMy research area is machine learning, reinforcement learning.\nAnd this semester I teach to other classes.\nHe s


#Step 2: Chunk the Text

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,   # larger chunks
    chunk_overlap=200,
    length_function=len
)

chunks = splitter.split_text(doc_text)
print(f"✅ Total chunks: {len(chunks)}")




✅ Total chunks: 16


#Step 3: Generate MCQs with Gemini

In [8]:
import re
import json

def generate_mcqs_with_gemini(chunk, num_questions=5):
    prompt = f"""
    You are an exam writer. Based ONLY on the passage below, create {num_questions} multiple-choice questions.

    Passage:
    \"\"\"{chunk}\"\"\"

    Requirements for each question:
    - Provide exactly 4 options (A, B, C, D).
    - Ensure only ONE option is correct.
    - Output valid JSON list only (no explanations, no text outside JSON).
    Example:
    [
      {{
        "question": "...",
        "choices": ["A ...","B ...","C ...","D ..."],
        "answer": "B"
      }}
    ]
    """

    model = genai.GenerativeModel("gemini-2.5-flash")
    response = model.generate_content(prompt)

    text = response.text.strip()
    match = re.search(r"\[.*\]", text, re.S)
    if match:
        return json.loads(match.group())
    else:
        raise ValueError("Gemini did not return valid JSON.")





#Step 4: Collect and Save MCQs

In [9]:
def validate_mcq(mcq, chunk):
    """
    Quality control:
    - Check if the correct answer text appears in the passage.
    - Return True if valid, False otherwise.
    """
    correct_answer = mcq.get("answer")
    if not correct_answer:
        return False
    # map "A"/"B"/"C"/"D" to text
    idx_map = {"A":0, "B":1, "C":2, "D":3}
    if correct_answer not in idx_map:
        return False
    correct_text = mcq["choices"][idx_map[correct_answer]]
    return correct_text.lower().split()[0] in chunk.lower()  # rough check


def classify_difficulty(question):
    """
    Ask Gemini to classify difficulty as Easy, Medium, Hard
    """
    prompt = f"""
    Classify the difficulty of the following question as Easy, Medium, or Hard
    (based on Bloom's Taxonomy level).
    Question: {question}
    Output only one word.
    """
    model = genai.GenerativeModel("gemini-2.5-flash")
    resp = model.generate_content(prompt)
    return resp.text.strip().split()[0]  # just first word


all_mcqs = []

for i, chunk in enumerate(chunks):
    try:
        mcqs = generate_mcqs_with_gemini(chunk, num_questions=5)  # user’s modification
        for j, mcq in enumerate(mcqs):
            mcq["id"] = f"q_{i+1}_{j+1}"
            mcq["source_preview"] = chunk[:120] + "..."

            # Quality Control filter
            if not validate_mcq(mcq, chunk):
                continue  # skip low-quality Q

            # Difficulty tag
            mcq["difficulty"] = classify_difficulty(mcq["question"])

            all_mcqs.append(mcq)
    except Exception as e:
        print(f"⚠️ Error on chunk {i}: {e}")

# Save results
with open("generated_mcqs.json", "w") as f:
    json.dump(all_mcqs, f, indent=2)

print("✅ Saved generated_mcqs.json with", len(all_mcqs), "high-quality questions")




✅ Saved generated_mcqs.json with 32 high-quality questions


#Preview Final Output

In [10]:
print(json.dumps(all_mcqs[:10], indent=2))



[
  {
    "question": "Who is teaching the class today, according to the passage?",
    "choices": [
      "A Professor Nathan",
      "B The keynote speaker",
      "C Delete Kiloton",
      "D An Associate Professor from another department"
    ],
    "answer": "C",
    "id": "q_1_1",
    "source_preview": "Substituting some shooting frozen RNN,\\nbut when does the clock\\nokay.\\nLet's start.\\nOkay. So yes.\\nWe will indeed disc...",
    "difficulty": "Easy"
  },
  {
    "question": "Why is Professor Nathan not present to teach the class?",
    "choices": [
      "A He is on sabbatical.",
      "B He is attending a research conference as a keynote speaker.",
      "C He is teaching another class this semester.",
      "D He is traveling for personal reasons."
    ],
    "answer": "B",
    "id": "q_1_2",
    "source_preview": "Substituting some shooting frozen RNN,\\nbut when does the clock\\nokay.\\nLet's start.\\nOkay. So yes.\\nWe will indeed disc...",
    "difficulty": "Medium"


In [11]:
from google.colab import files
files.download("generated_mcqs.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>